In [1]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

In [2]:
import findspark
findspark.init('spark-3.2.0-bin-hadoop3.2')

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('exercise').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/02 08:53:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df = spark.read.csv('data/walmart_stock.csv',inferSchema=True,header=True)

In [6]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [7]:
df.show()

+----------+------------------+------------------+------------------+------------------+--------+------------------+
|      Date|              Open|              High|               Low|             Close|  Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+--------+------------------+
|2012-01-03|         59.970001|         61.060001|         59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04|60.209998999999996|         60.349998|         59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05|         59.349998|         59.619999|         58.369999|         59.419998|12768200|         51.825539|
|2012-01-06|         59.419998|         59.450001|         58.869999|              59.0| 8069400|          51.45922|
|2012-01-09|         59.029999|         59.549999|         58.919998|             59.18| 6679300|51.616215000000004|
|2012-01-10|             59.43|59.709998999999996|             5

In [8]:
df.count()

1258

In [9]:
len(df.columns)

7

In [10]:
df.head(10)[0]

Row(Date='2012-01-03', Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996)

In [11]:
df.groupBy('Company').mean().show()

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



In [11]:
df.describe().show()

+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|      Date|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|      1258|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean|      null| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|      null|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|2012-01-03|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|2016-12-30|         90.800003|        90.970001|            89.25|        90.4700

In [12]:
df.describe().printSchema()

root
 |-- summary: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



In [13]:
from pyspark.sql.functions import format_number

In [14]:
result = df.describe()

In [19]:
result.select(result['summary'],format_number(result['Open'].cast('float'),2).alias('Open'),
             format_number(result['High'].cast('float'),2).alias('Hight'),
             format_number(result['Low'].cast('float'),2).alias('Low'),
             format_number(result['Low'].cast('float'),2).alias('Open'),
             format_number(result['Close'].cast('float'),2).alias('Open'),
             result['Volume'].cast('int').alias('Volume')
             ).show()


+-------+--------+--------+--------+--------+--------+--------+
|summary|    Open|   Hight|     Low|    Open|    Open|  Volume|
+-------+--------+--------+--------+--------+--------+--------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|1,258.00|    1258|
|   mean|   72.36|   72.84|   71.92|   71.92|   72.39| 8222093|
| stddev|    6.77|    6.77|    6.74|    6.74|    6.76| 4519780|
|    min|   56.39|   57.06|   56.30|   56.30|   56.42| 2094900|
|    max|   90.80|   90.97|   89.25|   89.25|   90.47|80898100|
+-------+--------+--------+--------+--------+--------+--------+



In [20]:
df2 = df.withColumn("HV Ratio",df["High"]/df["Volume"])

In [21]:
df2.select("HV Ratio").show()

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows



In [19]:
df.select(countDistinct('Sales')).show()

+---------------------+
|count(DISTINCT Sales)|
+---------------------+
|                   11|
+---------------------+



In [20]:
df.select(countDistinct('Sales').alias("average sales")).show()

+-------------+
|average sales|
+-------------+
|           11|
+-------------+



In [21]:
df.select(stddev('Sales')).show()

+------------------+
|stddev_samp(Sales)|
+------------------+
|250.08742410799007|
+------------------+



In [22]:
from pyspark.sql.functions import format_number

In [22]:
df.orderBy(df['High'].desc()).show()

+----------+-----------------+-----------------+-----------------+-----------------+--------+-----------------+
|      Date|             Open|             High|              Low|            Close|  Volume|        Adj Close|
+----------+-----------------+-----------------+-----------------+-----------------+--------+-----------------+
|2015-01-13|        90.800003|        90.970001|            88.93|        89.309998| 8215400|        83.825448|
|2015-01-08|        89.209999|90.66999799999999|            89.07|        90.470001|12713600|84.91421600000001|
|2015-01-09|            90.32|        90.389999|            89.25|        89.349998| 8522500|        83.862993|
|2015-01-12|        89.360001|        90.309998|        89.220001|        90.019997| 7372500|        84.491846|
|2015-01-23|88.41999799999999|        89.260002|        87.889999|        88.510002| 7565800|83.07458100000001|
|2015-01-26|        88.309998|        89.160004|        88.120003|        88.629997| 4666700|        83.

In [ ]:
from pyspark.sql.functions import mean
df.select(mean('Close')).show()

In [25]:
from pyspark.sql.functions import max,min
df.select(max('Volume'),min('Volume')).show()

+---------------------+
|format_number(std, 2)|
+---------------------+
|               250.09|
+---------------------+



In [ ]:
#How many days was the Close lower than 60 dollars?

In [27]:
df.filter('Close < 60').count()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+



In [ ]:
df.filter(df['Close'] < 60).count()